In [4]:
import pandas as pd
import csv as csv
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
df = pd.read_csv("train.csv").replace("male", 0).replace("female", 1)
df["Age"].head()
#欠損処理 by median
df["Age"].fillna(df.Age.median(), inplace = True)
df["Age"].head()
df["FamiliySize"] = df["SibSp"] + df["Parch"] + 1 #Familiysizeを親族＋子供の数＋本人で算出。同部屋の可能性などを探る。
df_dropped = df.drop(["Name", "SibSp", "Parch", "Ticket", "Fare", "Cabin", "Embarked", "PassengerId"], axis = 1)#関係ない奴、Familysizeに統合された奴は省いたDf作成。
df_dropped.head()

,Survived,Pclass,Sex,Age,FamiliySize
0,0,3,0,22.0,2
1,1,1,1,38.0,2
2,1,3,1,26.0,1
3,1,1,1,35.0,2
4,0,3,0,35.0,1


In [7]:
#パーセプトロンに引き続いて説明変数を二つ選んで格納していく。
#Sex and Pclassが高そうだから選ぶ
#今回はなぜか分からんがArray型に変換。DataFrameじゃ処理ができないっぽい。
x = df_dropped.loc[:,["Sex", "Pclass"]].values
y = df_dropped.loc[:,["Survived"]].values #Survivedの被説明変数を格納

# scikit-learnの仕様に合わせて､一列のベクトルに変換
y = y.reshape(-1)

# データの標準化、分類系では基本標準化した方が良い
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(x)
x_std = scaler.transform(x)
x_std

C:\Users\user\Anaconda3\lib\site-packages\sklearn\utils\validation.py:444: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


array([[-0.73769513,  0.82737724],
       [ 1.35557354, -1.56610693],
       [ 1.35557354,  0.82737724],
       ..., 
       [ 1.35557354,  0.82737724],
       [-0.73769513, -1.56610693],
       [-0.73769513,  0.82737724]])

In [12]:
#データ分割から学習へ
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x_std, y, test_size=0.3, random_state=0)
# ロジスティック回帰の実行
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(C=1.0)
lr.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [21]:
# 正答率を出力
print('train acc: %.3f' % lr.score(X_train, y_train))
print('test acc: %.3f' % lr.score(X_test, y_test))

train acc: 0.787
test acc: 0.787


In [23]:
#テストデータも同じ処理をしてreal_testを作成
df_test = pd.read_csv("test.csv").replace("male", 0).replace("female", 1)
#Sex and Pclassが高そうだから選ぶ
x_real_test = df_test.loc[:,["Sex", "Pclass"]]

# データの標準化、分類系では基本標準化した方が良い
scaler.fit(x_real_test)
x_test_std = scaler.transform(x_real_test)

#モデルにX_test_std入れて予測
output = lr.predict(x_test_std)

#予測結果とPassIDをZipする
zip_data = zip(df_test["PassengerId"].astype(int), output.astype(int))
predict_data = list(zip_data)

# テスト値を再読み込みして，rfでクラス分類したカラムを追加
df_out = pd.read_csv("./test.csv")
df_out["Survived"] = output

# outputディレクトリに出力する
df_out[["PassengerId","Survived"]].to_csv("./submission_LogReg.csv",index=False)